In [2]:
!pip install django
import os
import django

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "config.settings.develop")  # Proje adını doğru yaz
django.setup()



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = 'true'

In [3]:
import logging
logger = logging.getLogger('django.db.backends')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())


In [37]:
from medication_management.models import MedicationModel
from medication_management.models import MedicationReminderModel
from medication_management.models import MedicationUseHistoryModel 
from medication_management.models import StageSteps
from django.db import models

In [34]:
from datetime import date, datetime

In [11]:
mdc = MedicationModel.objects.get(id=2)



In [23]:
mdc.reminder.count()

2

In [25]:
hst_index = mdc.history.count()
hst_index

1

In [30]:
hst_index = mdc.history.count()
next_pilss = mdc.reminder.all().order_by('time')[hst_index if hst_index else 0 :]

In [33]:
next_pilss[0].time

datetime.time(23, 59, 54)

In [35]:
today = date.today()

In [38]:
medications = MedicationModel.objects.filter(
    # user=user,
    start_date__lte=today
).filter(
    models.Q(end_date__isnull=True) | models.Q(end_date__gte=today)
)
# implamente edermisin
hst_index = medications.history.count()
next_pilss = medications.reminder.all().order_by('time')[hst_index if hst_index else 0 :]

In [39]:
medications

<QuerySet [<MedicationModel: MedicationModel object (2)>]>

In [40]:
hst_index = medications.history.count()
next_pilss = medications.reminder.all().order_by('time')[hst_index if hst_index else 0 :]

AttributeError: 'QuerySet' object has no attribute 'history'

In [69]:
from datetime import date

today = date.today()
# user = request.user
results = []

medications = MedicationModel.objects.filter(
    # user=user,
    start_date__lte=today
).filter(
    models.Q(end_date__isnull=True) | models.Q(end_date__gte=today)
)
for med in medications:
    history_count = med.history.filter(use_date=today).count()
    reminders = med.reminder.all().order_by("time")
    next_reminders = reminders[history_count:]
    stage = mdc.stages.filter(
        range_start_date__lte=today
    ).filter(
        models.Q(range_end_date__isnull=True) | models.Q(range_end_date__gte=today)
    ).first()
    for reminder in next_reminders:
        results.append({
            "medication_name": med.name,
            "scheduled_time": reminder.time,
            "taken_count": history_count,
            "reminder_total": reminders.count(),
            "dosage": stage.unit,
            "dosage_type": stage.unit_type,
        })


<QuerySet [<MedicationModel: MedicationModel object (2)>, <MedicationModel: MedicationModel object (1)>]>


In [70]:
results

[{'medication_name': 'test 2',
  'scheduled_time': datetime.time(5, 54, 19),
  'taken_count': 0,
  'reminder_total': 2,
  'dosage': 2.0,
  'dosage_type': '23'},
 {'medication_name': 'test 2',
  'scheduled_time': datetime.time(20, 59, 54),
  'taken_count': 0,
  'reminder_total': 2,
  'dosage': 2.0,
  'dosage_type': '23'},
 {'medication_name': 'test',
  'scheduled_time': datetime.time(6, 54, 38),
  'taken_count': 0,
  'reminder_total': 1,
  'dosage': 2.0,
  'dosage_type': '23'}]

In [71]:
stage_steps = mdc.stages.filter(
    range_start_date__lte=today
).filter(
    models.Q(range_end_date__isnull=True) | models.Q(range_end_date__gte=today)
).first()

In [54]:
stage_steps

<StageSteps: StageSteps object (2)>

## almam gereken ilaçları bul 
bu kod kullanıcnın sadece o gün aldıkları ilaçlar göz önünde bulundurularak geriye kalan alması gereken ilaçları listeler

In [72]:
from datetime import date
from django.db.models import Q

today = date.today()
# user = request.user
results = []

medications = MedicationModel.objects.filter(
    Q(start_date__lte=today) &
    Q(end_date__isnull=True) | Q(end_date__gte=today)
    # & Q(user=user)
).prefetch_related("reminder", "history", "stages")

for med in medications:
    history_count = med.history.filter(use_date=today).count()
    reminders = sorted(med.reminder.all(), key=lambda r: r.time)
    next_reminders = reminders[history_count:]

    stage = med.stages.filter(
        Q(range_start_date__lte=today) &
        (Q(range_end_date__isnull=True) | Q(range_end_date__gte=today))
    ).order_by("range_start_date").first()

    for reminder in next_reminders:
        results.append({
            "medication_name": med.name,
            "scheduled_time": reminder.time,
            "taken_count": history_count,
            "reminder_total": len(reminders),
            "dosage": stage.unit if stage else None,
            "dosage_type": stage.unit_type if stage else None,
        })


In [73]:
results

[{'medication_name': 'test 2',
  'scheduled_time': datetime.time(5, 54, 19),
  'taken_count': 0,
  'reminder_total': 2,
  'dosage': 2.0,
  'dosage_type': '23'},
 {'medication_name': 'test 2',
  'scheduled_time': datetime.time(20, 59, 54),
  'taken_count': 0,
  'reminder_total': 2,
  'dosage': 2.0,
  'dosage_type': '23'},
 {'medication_name': 'test',
  'scheduled_time': datetime.time(6, 54, 38),
  'taken_count': 0,
  'reminder_total': 1,
  'dosage': None,
  'dosage_type': None}]